<div style="background-color: #0d0761; border-radius: 20px; color: #fff; font-weight: bold; padding: 10px; text-align:center">
    <h1>Pipeline de transformación completo con validaciones</h1>
</div>

<div style="font-weight: bold; color:#0d0761 ; border-width: 0 0 3px 0; border-style: solid; border-color: #0d0761; padding: 3px; ">
    <h2>Cargar librerías</h2>
</div>

In [1]:
import pandas as pd
import numpy as np

<div style="font-weight: bold; color:#0d0761 ; border-width: 0 0 3px 0; border-style: solid; border-color: #0d0761; padding: 3px; ">
    <h2>Crear dataset con datos que requieren transformación</h2>
</div>

In [2]:
np.random.seed(42)
n = 1000

df = pd.DataFrame({
    'id_cliente': range(1, n+1),
    'edad': np.random.normal(35, 15, n).clip(18, 80).astype(int),
    'ingresos': np.random.lognormal(10, 0.8, n),
    'gastos_mensuales': np.random.normal(2000, 500, n).clip(500, 10000),
    'categoria_cliente': np.random.choice(['A', 'B', 'C', 'D'], n),
    'fecha_registro': pd.date_range('2020-01-01', periods=n, freq='D')[:n],
    'email': [f'cliente{i}@ejemplo.com' for i in range(1, n+1)],
    'telefono': [f'({np.random.randint(100, 999)}){np.random.randint(100, 999)}-{np.random.randint(1000, 9999)}' for _ in range(n)]
})

# Introducir algunos errores intencionalmente
error_indices = np.random.choice(n, 50, replace=False)
df.loc[error_indices[:20], 'edad'] = np.random.choice([-5, 150, np.nan], 20)  # Edades inválidas
df.loc[error_indices[20:35], 'ingresos'] = -1000  # Ingresos negativos
df.loc[error_indices[35:], 'gastos_mensuales'] = df.loc[error_indices[35:], 'ingresos'] * 2  # Gastos > ingresos

In [3]:
# vistazo a las estadísticas de ['edad', 'ingresos', 'gastos_mensuales']

df.describe()[['edad', 'ingresos', 'gastos_mensuales']]

,edad,ingresos,gastos_mensuales
count,992.000000,1000.000000,1000.000000
mean,36.564516,31531.958648,2712.432462
min,-5.000000,-1000.000000,500.000000
25%,25.000000,13350.632142,1682.002341
50%,35.000000,22810.256741,2005.334057
75%,44.000000,38927.926463,2340.766325
max,150.000000,283363.570715,116643.085166
std,17.191621,29332.952117,7200.577985


In [4]:
print(f'Valores ausentes:\n{df.isna().sum()[df.isna().sum()>0]}')

Valores ausentes:
edad    8
dtype: int64


<div style="font-weight: bold; color:#0d0761 ; border-width: 0 0 3px 0; border-style: solid; border-color: #0d0761; padding: 3px; ">
    <h2>Aplicar validaciones y correcciones</h2>
</div>

In [5]:
# Validar y corregir edades
df['edad_valida'] = df['edad'].apply(lambda x: True if 18 <= x <= 80 else False)
df.loc[~df['edad_valida'], 'edad'] = np.nan  # Marcar inválidas como NaN

# Validar ingresos (no negativos)
df.loc[df['ingresos'] < 0, 'ingresos'] = np.nan

# Validar gastos vs ingresos
df['ratio_gasto_ingreso'] = df['gastos_mensuales'] / df['ingresos']

# ajuste para evitar gastos mayores a los ingresos
df.loc[df['ratio_gasto_ingreso'] > 1, 'gastos_mensuales'] = df.loc[df['ratio_gasto_ingreso'] > 1, 'ingresos'] * 0.8

Es interesante el ajuste efectuado para los gastos mensuales: se actualiza condicionalmente los gastos_mensuales a ser el 80% de los ingresos solo para aquellos registros donde el ratio inicial era superior a 1. Esto claramente solo sirve para facilitar el análisis de los datos en este ejercicio, pues en un escenario real no tendría sentido.

Una consideración adicional que habría que efectuar en un escenario real sería el manejo de los datos cuando los ingresos fuesen nulos, porque pandas arrojaría un valor inf para el ratio si los gastos fuesen mayores a cero.

In [6]:
# vistazo a las estadísticas de ['edad', 'ingresos', 'gastos_mensuales']

df.describe()[['edad', 'ingresos', 'gastos_mensuales']]

,edad,ingresos,gastos_mensuales
count,980.000000,985.000000,1000.000000
mean,35.650000,32027.369186,2267.924653
min,18.000000,2095.796796,500.000000
25%,25.000000,13605.512925,1682.002341
50%,35.000000,23192.391049,2005.334057
75%,44.000000,39422.958748,2340.250978
max,80.000000,283363.570715,46657.234066
std,13.219331,29277.287020,2800.730403


In [7]:
print(f'Valores ausentes:\n{df.isna().sum()[df.isna().sum()>0]}')

Valores ausentes:
edad                   20
ingresos               15
ratio_gasto_ingreso    15
dtype: int64


Con las correcciones:
- el número de valores ausentes para la columna "edad" aumentó de 8 a 20, y el de columna "ingresos" aumentó de 0 a 15, como era de esperar, y
- el número de valores ausentes para la columna "ratio_gasto_ingreso" coincide con la de "ingresos" porque no hay valores ausentes en la columna "gastos_mensuales" (por la construcción y modificaciones efectuadas al dataset).

<div style="font-weight: bold; color:#0d0761 ; border-width: 0 0 3px 0; border-style: solid; border-color: #0d0761; padding: 3px; ">
    <h2>Crear transformaciones y enriquecimientos</h2>
</div>

In [8]:
# Categorizar por edad
df['grupo_edad'] = pd.cut(df['edad'], 
                        bins=[18, 25, 35, 50, 80], 
                        labels=['Joven', 'Adulto_Joven', 'Adulto', 'Senior'], include_lowest=True)

# Calcular capacidad de ahorro
df['capacidad_ahorro'] = df['ingresos'] - df['gastos_mensuales']
df['ratio_ahorro'] = df['capacidad_ahorro'] / df['ingresos']

# Clasificar capacidad financiera
df['clasificacion_financiera'] = np.where(df['ratio_ahorro'] > 0.3, 'Ahorra_Mucho',
                                         np.where(df['ratio_ahorro'] > 0.1, 'Ahorra_Poco',
                                                 np.where(df['ratio_ahorro'] > 0, 'Equilibra', 'Deficit')))

# Extraer información del teléfono
df['codigo_area'] = df['telefono'].str.extract(r'\((\d{3})\)')

# Calcular antigüedad
df['antiguedad_dias'] = (pd.Timestamp.now() - df['fecha_registro']).dt.days
df['antiguedad_meses'] = df['antiguedad_dias'] // 30

Se incorporó el parámetro include_lowest=True en pd.cut para que categorice correctamente a los individuos de 18 años en el segmento "Joven".

Hay un problema con la definición de clasificación financiera de "Deficit", pues ese valor se está asignando a individuos cuyos ingresos son desconocidos. La clasificación sería correcta si los gastos fuesen mayores a los ingresos, pero eso no se tiene en este dataframe (por las correcciones efectuadas). 

In [9]:
print(f"Valores únicos de ingresos cuando la clasificación financiera es \"Deficit\" = {df.loc[df.clasificacion_financiera=='Deficit', 'ingresos'].unique()}")

Valores únicos de ingresos cuando la clasificación financiera es "Deficit" = [nan]


Se propone recalcular la clasificación financiera de la siguiente manera:

In [10]:
df['clasificacion_financiera_modificada'] = np.where(df['ratio_ahorro'] > 0.3,   'Ahorra_Mucho',
                                            np.where(df['ratio_ahorro'] > 0.1,   'Ahorra_Poco',
                                            np.where(df['ratio_ahorro'] >= 0,    'Equilibra', 
                                            np.where(df['ratio_ahorro'].notna(), 'Deficit', 'Indeterminable'))))

In [11]:
# Vistazo al dataframe para ver aquellos registros en que hay diferencias entre la clasificación financiera original y la modificada.

mask = df.clasificacion_financiera != df.clasificacion_financiera_modificada
df.loc[mask, ['ingresos', 'gastos_mensuales', 'capacidad_ahorro', 'ratio_ahorro', 'clasificacion_financiera', 'clasificacion_financiera_modificada']]

,ingresos,gastos_mensuales,capacidad_ahorro,ratio_ahorro,clasificacion_financiera,clasificacion_financiera_modificada
99,NaN,2408.944829,NaN,NaN,Deficit,Indeterminable
232,NaN,1014.948091,NaN,NaN,Deficit,Indeterminable
363,NaN,2556.344187,NaN,NaN,Deficit,Indeterminable
525,NaN,2241.343944,NaN,NaN,Deficit,Indeterminable
539,NaN,2027.467055,NaN,NaN,Deficit,Indeterminable
643,NaN,1673.955556,NaN,NaN,Deficit,Indeterminable
644,NaN,1455.183481,NaN,NaN,Deficit,Indeterminable
761,NaN,1597.842120,NaN,NaN,Deficit,Indeterminable
765,NaN,2410.172006,NaN,NaN,Deficit,Indeterminable
767,NaN,2227.370456,NaN,NaN,Deficit,Indeterminable


Vemos que se corrigieron todos los registros donde la clasificación financiera original señalaba déficit.

In [12]:
# Vistazo a la extracción del código de área

df[['telefono', 'codigo_area']].head()

,telefono,codigo_area
0,(138)588-4715,138
1,(641)769-2301,641
2,(302)165-3155,302
3,(367)321-1692,367
4,(354)554-7396,354


El código de área se extrae correctamente.

In [13]:
# Vistazo a la antigüedad

df[['fecha_registro', 'antiguedad_dias', 'antiguedad_meses']].head()

,fecha_registro,antiguedad_dias,antiguedad_meses
0,2020-01-01,2153,71
1,2020-01-02,2152,71
2,2020-01-03,2151,71
3,2020-01-04,2150,71
4,2020-01-05,2149,71


Los datos se generan se acuerdo a la regla escrita.

<div style="font-weight: bold; color:#0d0761 ; border-width: 0 0 3px 0; border-style: solid; border-color: #0d0761; padding: 3px; ">
    <h2>Crear métricas agregadas por categoría</h2>
</div>

In [14]:
# Métricas por grupo de edad
metricas_edad = df.groupby('grupo_edad', observed=True).agg({
    'ingresos': ['mean', 'median', 'std'],
    'capacidad_ahorro': 'mean',
    'ratio_ahorro': 'mean'
}).round(2)

print("Métricas por grupo de edad:")
print(metricas_edad)

# Resumen de validaciones
resumen_validacion = {
    'total_registros': len(df),
    'edades_invalidas': (~df['edad_valida']).sum(),
    'ingresos_negativos_corregidos': (df['ingresos'].isna()).sum(),
    'registros_procesados': len(df)
}

print("\nResumen de validación:")
for clave, valor in resumen_validacion.items():
    print(f"{clave}: {valor}")

Métricas por grupo de edad:
              ingresos                     capacidad_ahorro ratio_ahorro
                  mean    median       std             mean         mean
grupo_edad                                                              
Joven         33090.02  26240.77  26168.85         30704.12         0.88
Adulto_Joven  31465.15  23640.71  25554.76         29336.67         0.87
Adulto        31504.19  21448.63  32238.73         29145.95         0.87
Senior        31016.96  22286.82  26921.09         28876.55         0.88

Resumen de validación:
total_registros: 1000
edades_invalidas: 20
ingresos_negativos_corregidos: 15
registros_procesados: 1000


Se cambió df.groupby('grupo_edad') por df.groupby('grupo_edad', observed=True) para evitar warning: *"FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning. metricas_edad = df.groupby('grupo_edad').agg({"*

Las métricas están calculadas correctamente. Recordar que se efectuó una modificación al código para que el segmento "Joven" incluyese a los individuos de 18 años.

Podría resultar interesante incorporar en las métricas el número de individuos analizados dentro de cada segmento.

In [15]:
# Métricas por grupo de edad
metricas_edad = df.groupby('grupo_edad', observed=True).agg({
    'ingresos': ['mean', 'median', 'std'],
    'capacidad_ahorro': 'mean',
    'ratio_ahorro': 'mean',
    'grupo_edad': 'count'
}).round(2)

print("Métricas por grupo de edad:")
display(metricas_edad)

Métricas por grupo de edad:


ingresos                     capacidad_ahorro ratio_ahorro  \
                  mean    median       std             mean         mean   
grupo_edad                                                                 
Joven         33090.02  26240.77  26168.85         30704.12         0.88   
Adulto_Joven  31465.15  23640.71  25554.76         29336.67         0.87   
Adulto        31504.19  21448.63  32238.73         29145.95         0.87   
Senior        31016.96  22286.82  26921.09         28876.55         0.88   

             grupo_edad  
                  count  
grupo_edad               
Joven               259  
Adulto_Joven        249  
Adulto              336  
Senior              136